In [432]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import glob
from pathlib import Path
import matplotlib.pyplot as plt
import random
from scipy.fft import fft, ifft

In [433]:
import sys
sys.path.append('C:/Users/DimiP/Documents/GitHub/ULG/COURS/basic-MPC/src')
from features import process_utils
%reload_ext autoreload

In [434]:
import importlib
importlib.reload(process_utils)

<module 'features.process_utils' from 'C:\\Users/DimiP/Documents/GitHub/ULG/COURS/basic-MPC/src\\features\\process_utils.py'>

In [435]:
#création des dicts de dataframe
path='c:\\Users\\DimiP\\Documents\\GitHub\\ULG\\COURS\\basic-MPC\\data\\raw'
csv_files = glob.glob(os.path.join(path, "*.csv"))
data={}
for file in csv_files:
    
    p=Path(file)
    filename=p.with_suffix('').stem
    finalfilename=filename.replace("temperature_", "")
    data[finalfilename]=pd.read_csv(p)
    data[finalfilename].index=data[finalfilename]['time'].apply(lambda x: process_utils.convertdate(x))


    

In [436]:
data['pv_production_load'].columns = ['time', 'L1_PV', 'L2_PV', 'L3_PV', 'L1_Load', 'L2_Load','L3_Load']

In [437]:
pv=data['pv_production_load'].to_numpy()

In [438]:
#dataframe avec index seul

big_df = pd.DataFrame(index=data['kitchen'].index)
#big_df['total sun'] = np.abs(pv[:,1])+np.abs(pv[:,2])+np.abs(pv[:,3])
#big_df['pv load'] = np.abs(pv[:,4])+np.abs(pv[:,5])+np.abs(pv[:,6])

In [203]:
#data['pv_production_load']['datetime_obj']=data['pv_production_load'].apply(lambda x: process_utils.infodate(x['time'])[0],axis=1)


In [439]:
date_array = [process_utils.datetimeobj(x) for x in big_df.index]


In [440]:
df_add = pd.DataFrame(data=date_array,columns=['perc_D','week_D','number_D','year'],index=big_df.index)

In [441]:
big_df = pd.concat([big_df,df_add], axis=1)

In [442]:
mesT_df=pd.concat([data[key]['current_value'].dropna() for key in ['bathroom', 'bedroom_1', 'bedroom_2', 'bedroom_3',
 'diningroom', 'kitchen', 'livingroom']],axis=1)
setT_df=pd.concat([data[key]['setpoint'] for key in ['bathroom', 'bedroom_1', 'bedroom_2', 'bedroom_3',
 'diningroom', 'kitchen', 'livingroom']],axis=1)

In [443]:
big_df['multizone_mesT']=mesT_df.apply(list,axis=1)
big_df['multizone_setT']=setT_df.apply(list,axis=1)
big_df['outside_mesT']=data['outside']['current_value']
finaldf=big_df[big_df.index>pd.to_datetime('2020-05-24 19:40:03')]


In [444]:
def sum_nonneg(x):
    return sum(max(i,0) for i in x)
#data['pv_production_load']['PV'] = data['pv_production_load'][['L1_PV','L2_PV','L3_PV']].apply(lambda x: sum_nonneg(list(x)))
#data['pv_production_load']['PV_load'] = data['pv_production_load'][['L1_Load','L2_Load','L3_Load']].apply(lambda x: sum_nonneg(list(x)))


In [446]:
pv_df=data['pv_production_load'].reindex(index=big_df.index,method='nearest')
#[data['pv_production_load'].index > big_df.index.min() ]

In [447]:
pv_df.drop_duplicates(subset='time')
pv_df['PV'] = pv_df[['L1_PV','L2_PV','L3_PV']].apply(lambda x: sum_nonneg(x),axis=1)
pv_df['PV_load'] = pv_df[['L1_Load','L2_Load','L3_Load']].apply(lambda x: sum_nonneg(x),axis=1)

In [449]:
def averageT(Tlist):
    return sum(Tlist) / len(Tlist)
big_df['singlezone_mesT'] = big_df['multizone_mesT'].apply(lambda x : averageT(x))
big_df['singlezone_setT'] = big_df['multizone_setT'].apply(lambda x : averageT(x))

In [451]:
lst=big_df.index
big_df['singlezone_nextT'] = pd.Series()
big_df['multizone_nextT'] = pd.Series()
big_df['singlezone_nextT']=big_df['singlezone_mesT'].shift(-1)
big_df['multizone_nextT'] = big_df['multizone_mesT'].shift(-1)


emptydf=pd.DataFrame(index=big_df.index)
emptydf['timestamp']=big_df.index
big_df['timestep']=emptydf['timestamp'].diff().dt.total_seconds().shift(-1)


C:\Users\DimiP\AppData\Local\Temp/ipykernel_14344/2026541694.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  big_df['singlezone_nextT'] = pd.Series()
C:\Users\DimiP\AppData\Local\Temp/ipykernel_14344/2026541694.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  big_df['multizone_nextT'] = pd.Series()


In [458]:
big_df=big_df.dropna()
big_df.head(5)

,perc_D,week_D,number_D,year,multizone_mesT,multizone_setT,outside_mesT,singlezone_mesT,singlezone_setT,singlezone_nextT,multizone_nextT,timestep
time,,,,,,,,,,,,
2020-05-24 19:40:03,0.819479,sunday,145,2020,"[21.8, 22.5, 22.4, 21.7, 22.0, 22.0, 22.9]","[16.0, 16.0, 16.0, 15.0, 16.0, 16.0, 16.0]",14.0,22.185714,15.857143,22.185714,"[21.8, 22.5, 22.4, 21.7, 22.0, 22.0, 22.9]",300.0
2020-05-24 19:45:03,0.822951,sunday,145,2020,"[21.8, 22.5, 22.4, 21.7, 22.0, 22.0, 22.9]","[16.0, 16.0, 16.0, 15.0, 16.0, 16.0, 16.0]",14.0,22.185714,15.857143,22.128571,"[21.8, 22.4, 22.4, 21.7, 21.7, 22.0, 22.9]",300.0
2020-05-24 19:50:03,0.826424,sunday,145,2020,"[21.8, 22.4, 22.4, 21.7, 21.7, 22.0, 22.9]","[16.0, 16.0, 16.0, 15.0, 16.0, 16.0, 16.0]",14.0,22.128571,15.857143,22.128571,"[21.8, 22.4, 22.4, 21.7, 21.7, 22.0, 22.9]",300.0
2020-05-24 19:55:03,0.829896,sunday,145,2020,"[21.8, 22.4, 22.4, 21.7, 21.7, 22.0, 22.9]","[16.0, 16.0, 16.0, 15.0, 16.0, 16.0, 16.0]",14.0,22.128571,15.857143,22.128571,"[21.8, 22.4, 22.4, 21.7, 21.7, 22.0, 22.9]",301.0
2020-05-24 20:00:04,0.833380,sunday,145,2020,"[21.8, 22.4, 22.4, 21.7, 21.7, 22.0, 22.9]","[16.0, 16.0, 16.0, 15.0, 16.0, 16.0, 16.0]",14.0,22.128571,15.857143,22.128571,"[21.8, 22.4, 22.4, 21.7, 21.7, 22.0, 22.9]",300.0


In [459]:
big_df.to_csv('C:/Users/DimiP/Documents/GitHub/ULG/COURS/basic-MPC/data/raw/big_df.csv')

In [461]:
data['livingroom'].isna().sum()

time             0
current_value    0
setpoint         0
dtype: int64